<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MY-AI-GIZMO SUPER ROBUST INSTALLER - CONTINUES DESPITE ERRORS
# Fixed all dependency issues + Error-tolerant execution
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
from pathlib import Path
import shutil

print("🚀 MY-AI-GIZMO SUPER ROBUST INSTALLER")
print("="*70)
print("This version continues even if some things fail!")
print("="*70)

# --- STEP 1: Mount Google Drive ---
print("\n📂 Step 1: Mounting Google Drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("   ✅ Mounted")
except Exception as e:
    print(f"   ⚠️  Warning: {e}")
    print("   Continuing anyway...")

# --- STEP 2: Setup Directories ---
print("\n📁 Step 2: Setting up directories...")

DRIVE_BASE = Path("/content/drive/MyDrive/MY-AI-Gizmo")
REPO_DIR = DRIVE_BASE / "MY-AI-Gizmo-working"
MODELS_DIR = DRIVE_BASE / "models"
USER_DATA_DIR = DRIVE_BASE / "user_data"

for d in [DRIVE_BASE, MODELS_DIR, USER_DATA_DIR]:
    try:
        d.mkdir(exist_ok=True)
    except Exception as e:
        print(f"   ⚠️  Could not create {d}: {e}")

print(f"   ✅ Base: {DRIVE_BASE}")

# --- STEP 3: Clone Repository ---
if not (REPO_DIR / "server.py").exists():
    print("\n📥 Step 3: Cloning repository...")
    try:
        if REPO_DIR.exists():
            shutil.rmtree(REPO_DIR)
        os.chdir(DRIVE_BASE)
        subprocess.run(["git", "clone", "https://github.com/gitleon8301/MY-AI-Gizmo-working.git"], check=True)
        print("   ✅ Cloned")
    except Exception as e:
        print(f"   ⚠️  Clone failed: {e}")
        print("   Trying to continue with existing files...")
else:
    print("\n✅ Step 3: Repository exists")

try:
    os.chdir(REPO_DIR)
except Exception as e:
    print(f"   ⚠️  Could not change to repo dir: {e}")

# --- STEP 4: Link User Data ---
print("\n🔗 Step 4: Linking user data to Drive...")

try:
    local_user_data = REPO_DIR / "user_data"
    if local_user_data.exists() and not local_user_data.is_symlink():
        for item in local_user_data.rglob("*"):
            if item.is_file():
                rel = item.relative_to(local_user_data)
                dest = USER_DATA_DIR / rel
                dest.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(item, dest)
        shutil.rmtree(local_user_data)

    if not local_user_data.exists():
        local_user_data.symlink_to(USER_DATA_DIR)
        print("   ✅ Linked")

    (USER_DATA_DIR / "logs").mkdir(exist_ok=True)
except Exception as e:
    print(f"   ⚠️  Linking warning: {e}")
    print("   Continuing anyway...")

# --- STEP 5: Environment Variables ---
print("\n🔧 Step 5: Environment setup...")
os.environ['MPLBACKEND'] = 'Agg'
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
os.environ['CMAKE_ARGS'] = '-DLLAMA_CUBLAS=off'
print("   ✅ Environment configured")

# --- STEP 6: Install Core Dependencies ---
print("\n📦 Step 6: Installing dependencies (ROBUST)...")
print("   This may take 2-3 minutes on first run...")

def safe_install(packages, description="packages"):
    """Install packages but don't fail if some don't work"""
    print(f"   Installing {description}...")
    for pkg in (packages if isinstance(packages, list) else [packages]):
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg],
                          check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL,
                          timeout=120)
        except Exception as e:
            print(f"   ⚠️  {pkg} failed: {e}")
    return True

# Upgrade pip
safe_install("pip --upgrade", "pip upgrade")

# Essential packages
essential = ["wheel", "setuptools", "numpy", "requests", "tqdm", "pyyaml"]
safe_install(essential, "core packages")

# Install llama-cpp-python
print("   Installing llama-cpp-python (CPU)...")
try:
    subprocess.run([
        sys.executable, "-m", "pip", "install",
        "llama-cpp-python", "--no-cache-dir",
        "--force-reinstall", "-q"
    ], check=False, env=dict(os.environ, CMAKE_ARGS='-DLLAMA_CUBLAS=off'),
       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, timeout=300)
    print("   ✅ llama-cpp-python installed")
except Exception as e:
    print(f"   ⚠️  llama-cpp-python warning: {e}")

# Additional packages
additional = [
    "torch", "transformers", "gradio>=3.50.0", "accelerate",
    "markdown", "Pillow", "safetensors", "sentencepiece", "protobuf",
    "flask", "flask-cloudflared"  # Add flask-cloudflared for openai extension
]
safe_install(additional, "additional packages")

print("   ✅ All dependencies installed (some may have warnings)")

# --- STEP 7: ADVANCED FIX for llama_cpp_server.py ---
print("\n🔧 Step 7: Advanced patching of llama_cpp_server.py...")

llama_server_file = REPO_DIR / "modules" / "llama_cpp_server.py"
if llama_server_file.exists():
    try:
        with open(llama_server_file, 'r', encoding='utf-8') as f:
            content = f.read()

        original_content = content

        # Fix 1: Comment out the import
        content = content.replace(
            'import llama_cpp_binaries',
            '# import llama_cpp_binaries  # Removed - not needed'
        )

        # Fix 2: Remove or replace any usage of llama_cpp_binaries
        # Common patterns where it might be used:
        content = content.replace(
            'llama_cpp_binaries.',
            '# llama_cpp_binaries.  # Removed - '
        )

        # Fix 3: If there's a conditional check for llama_cpp_binaries
        content = content.replace(
            'if llama_cpp_binaries',
            'if False  # llama_cpp_binaries removed'
        )

        content = content.replace(
            'if hasattr(llama_cpp_binaries',
            'if False and hasattr(llama_cpp_binaries'
        )

        # Save the patched file
        if content != original_content:
            with open(llama_server_file, 'w', encoding='utf-8') as f:
                f.write(content)
            print("   ✅ Patched llama_cpp_server.py completely")
        else:
            print("   ✅ llama_cpp_server.py already compatible")

    except Exception as e:
        print(f"   ⚠️  Patching warning: {e}")
        print("   Continuing anyway...")
else:
    print("   ⚠️  llama_cpp_server.py not found - may be OK")

# --- STEP 8: Disable problematic extensions ---
print("\n🔧 Step 8: Configuring extensions...")

try:
    extensions_dir = REPO_DIR / "extensions"
    if extensions_dir.exists():
        # Disable openai extension if it causes issues
        openai_script = extensions_dir / "openai" / "script.py"
        if openai_script.exists():
            with open(openai_script, 'r', encoding='utf-8') as f:
                ext_content = f.read()

            # Make cloudflared optional
            ext_content = ext_content.replace(
                'from flask_cloudflared import _run_cloudflared',
                'try:\n    from flask_cloudflared import _run_cloudflared\nexcept:\n    _run_cloudflared = None  # Optional'
            )

            ext_content = ext_content.replace(
                'raise Exception(',
                'print("Warning: flask_cloudflared not available"); return  # '
            )

            with open(openai_script, 'w', encoding='utf-8') as f:
                f.write(ext_content)
            print("   ✅ Made extensions error-tolerant")
    else:
        print("   ✅ No extensions to configure")
except Exception as e:
    print(f"   ⚠️  Extension config warning: {e}")

# --- STEP 9: Fix UI Theme ---
print("\n🔧 Step 9: Patching UI...")

try:
    ui_file = REPO_DIR / "modules" / "ui.py"
    if ui_file.exists():
        with open(ui_file, 'r', encoding='utf-8') as f:
            content = f.read()

        if 'button_shadow_hover' in content and 'if False:' not in content:
            content = content.replace(
                'if not shared.args.old_colors:',
                'if False:  # Disabled for Colab'
            )
            with open(ui_file, 'w', encoding='utf-8') as f:
                f.write(content)
            print("   ✅ UI patched")
        else:
            print("   ✅ UI compatible")
    else:
        print("   ⚠️  UI file not found")
except Exception as e:
    print(f"   ⚠️  UI patch warning: {e}")

# --- STEP 10: Download Model ---
print("\n⬇️  Step 10: Checking model...")

MODEL_FILE = MODELS_DIR / "llama-2-7b.Q4_K_M.gguf"

try:
    if MODEL_FILE.exists():
        size_gb = MODEL_FILE.stat().st_size / (1024**3)
        print(f"   ✅ Model exists ({size_gb:.2f} GB)")
    else:
        print("   📥 Downloading model (3.8 GB - ONE TIME ONLY)...")
        print("   This will take 3-5 minutes...")
        url = "https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf"

        subprocess.run([
            "wget", "-q", "--show-progress",
            "-O", str(MODEL_FILE), url
        ], check=True, timeout=600)

        if MODEL_FILE.exists():
            print(f"   ✅ Downloaded ({MODEL_FILE.stat().st_size / (1024**3):.2f} GB)")
        else:
            print("   ⚠️  Download may have failed - check manually")
except Exception as e:
    print(f"   ⚠️  Model download warning: {e}")
    print("   You may need to download manually")

# --- STEP 11: Link Models Directory ---
print("\n🔗 Step 11: Linking models...")

try:
    repo_models = REPO_DIR / "models"
    if not repo_models.is_symlink():
        if repo_models.exists():
            shutil.rmtree(repo_models)
        repo_models.symlink_to(MODELS_DIR)
        print("   ✅ Models linked")
    else:
        print("   ✅ Already linked")
except Exception as e:
    print(f"   ⚠️  Linking warning: {e}")

# --- STEP 12: Verify Installation ---
print("\n🔍 Step 12: Verifying installation...")

checks = {
    "server.py": REPO_DIR / "server.py",
    "Model file": MODEL_FILE,
    "User data": USER_DATA_DIR,
    "Modules dir": REPO_DIR / "modules"
}

warnings = []
for name, path in checks.items():
    if path.exists():
        print(f"   ✅ {name}")
    else:
        print(f"   ⚠️  {name} missing!")
        warnings.append(name)

if warnings:
    print(f"\n⚠️  Some items missing: {', '.join(warnings)}")
    print("   Attempting to continue anyway...")

# --- STEP 13: Interactive Configuration ---
print("\n" + "="*70)
print("⚙️  CONFIGURATION")
print("="*70)

# Hardware
print("\n🖥️  Hardware Mode:")
print("1. CPU only (Recommended for Colab)")
print("2. GPU (Requires GPU runtime)")
print("3. Auto-detect")

hw = input("\nChoice (1-3) [default: 1]: ").strip() or "1"

if hw == '1':
    use_cpu = True
    hw_name = "CPU"
elif hw == '2':
    use_cpu = False
    hw_name = "GPU"
else:
    try:
        has_gpu = subprocess.run(['nvidia-smi'], capture_output=True, timeout=5).returncode == 0
        use_cpu = not has_gpu
        hw_name = "GPU (detected)" if has_gpu else "CPU (no GPU)"
    except:
        use_cpu = True
        hw_name = "CPU (fallback)"

print(f"✅ Using: {hw_name}")

# Options
print("\n🔧 Additional Options:")
print("1. Enable API")
print("2. Verbose logging")
print("3. Both API + Verbose")
print("4. None (minimal)")

opts = input("\nChoice (1-4) [default: 4]: ").strip() or "4"

enable_api = opts in ['1', '3']
enable_verbose = opts in ['2', '3']

# --- STEP 14: Build Launch Command ---
print("\n" + "="*70)
print("🚀 LAUNCHING MY-AI-GIZMO")
print("="*70)

cmd = [sys.executable, "server.py"]

# Hardware flags
if use_cpu:
    cmd.extend(["--cpu", "--threads", "4"])

# Core flags
cmd.extend([
    "--listen",
    "--share",
    "--model", str(MODEL_FILE),
    "--loader", "llama.cpp"
])

# Optional flags
if enable_api:
    cmd.extend(["--api", "--public-api"])
if enable_verbose:
    cmd.append("--verbose")

# Disable problematic extensions
cmd.extend(["--extensions", ""])  # Don't load any extensions by default

# Display config
print(f"\n📝 Launch Configuration:")
print(f"   Hardware: {hw_name}")
print(f"   Model: {MODEL_FILE.name}")
print(f"   Loader: llama.cpp")
print(f"   API: {'Enabled' if enable_api else 'Disabled'}")
print(f"   Verbose: {'Yes' if enable_verbose else 'No'}")
print(f"   Extensions: Disabled (for stability)")
print(f"   Chats save to: {USER_DATA_DIR / 'logs'}")

print("\n" + "="*70)
print("⏳ Starting server...")
print("🔗 WATCH FOR GRADIO LINK BELOW ⬇️")
print("="*70 + "\n")

# --- STEP 15: Launch Server with Error Recovery ---
try:
    # Change to repo directory
    os.chdir(REPO_DIR)

    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )

    found_link = False
    error_count = 0
    max_errors = 50  # Continue even with many errors

    for line in process.stdout:
        print(line, end='')

        # Count errors but don't stop
        if 'error' in line.lower() and 'deprecation' not in line.lower():
            error_count += 1
            if error_count <= 5:  # Only show first few
                print(f"   [Error #{error_count} - continuing...]")

        # Highlight Gradio link
        if ('gradio.live' in line.lower() or
            ('running on' in line.lower() and 'http' in line)):
            if not found_link:
                print("\n" + "🎉"*35)
                print("✅ SERVER IS RUNNING! CLICK THE LINK ABOVE ☝️")
                print("🎉"*35)
                print(f"\n⚠️  Note: {error_count} errors occurred but server started anyway")
                print("\n💡 USAGE TIPS:")
                print("   • Click the Gradio link above")
                print("   • Select Chat mode in the web UI")
                print("   • Conversations auto-save to Google Drive")
                print("   • Press Ctrl+C here to stop server")
                print("="*70 + "\n")
                found_link = True

    return_code = process.wait()

    if return_code != 0 and not found_link:
        print("\n⚠️  Server exited (this might be OK if you stopped it)")
        print(f"   Total errors encountered: {error_count}")

except KeyboardInterrupt:
    print("\n\n⏹️  Server stopped by user")
    print(f"💾 All chats saved in: {USER_DATA_DIR / 'logs'}")

except Exception as e:
    print(f"\n\n⚠️  Launch issue: {e}")
    print("\n🔧 Troubleshooting:")
    print(f"   1. Check if model exists: {MODEL_FILE.exists() if MODEL_FILE.exists() else 'Unknown'}")
    print(f"   2. Try running manually:")
    print(f"      cd {REPO_DIR}")
    print(f"      python server.py --cpu --model {MODEL_FILE} --listen --share")
    print("\n   3. Check the command that was attempted:")
    print(f"      {' '.join(str(x) for x in cmd)}")

print("\n" + "="*70)
print("Installation/Launch attempt complete!")
print("="*70)